In [1]:

class Identifiers:
    MAJOR = ("major", "big")
    MINOR = ("minor", "small")
    PATCH = ("patch", "fix")
    
    @classmethod
    def names(cls) -> tuple[str, ...]:
        return cls.MAJOR + cls.MINOR + cls.PATCH
    
    @classmethod 
    def names_repr(cls) -> str:
        return ", ".join(list(map(repr, cls.names())))
    
    

def update_version_info(version_info: list[int], identifier: str) -> list[int]:    
    if not identifier in Identifiers.names():
        raise ValueError(
            f"Expected one of {Identifiers.names_repr()}. Got: {identifier.lower()!r}"
        )
        
    if identifier.lower() in Identifiers.MAJOR:
        version_info[0] += 1
        version_info[1] = 0
        version_info[2] = 0
    elif identifier.lower() in Identifiers.MINOR:
        version_info[1] += 1
        version_info[2] = 0
    elif identifier.lower() in Identifiers.PATCH:
        version_info[2] += 1
    

    return version_info

update_version_info([1, 2, 3], "fix") 


[1, 2, 4]

In [17]:
from version_updater2._version import __version_info__ as version_info

print(version_info)
new_version_info = update_version_info(version_info.copy(), "fix")
print(new_version_info)

[1, 0, 0]
[1, 0, 1]


In [42]:
from pathlib import Path

current_content = Path("version_updater2/_version.py").read_text()

def info2version(info: list[int])-> str: 
    return ".".join(list(map(str, info)))

version = info2version(version_info.copy())
new_version = info2version(new_version_info)

new_content = current_content.replace(
f'__version__ = "{version}"',
f'__version__ = "{new_version}"'
)

Path("version_updater2/_version.py").write_text(new_content)



80

In [96]:
import re
from dataclasses import dataclass
from pathlib import Path
import subprocess


class Identifiers:
    MAJOR = ("major", "big")
    MINOR = ("minor", "small")
    PATCH = ("patch", "fix")

    @classmethod
    def names(cls) -> tuple[str, ...]:
        return cls.MAJOR + cls.MINOR + cls.PATCH

    @classmethod
    def names_repr(cls) -> str:
        return ", ".join(list(map(repr, cls.names())))


@dataclass
class MultipleFilesFoundError(OSError):
    files: list[Path]

    def __str__(self):
        files = ", ".join(map(str, self.files))
        return f"Found more than one file with the same name: {files}"


def find_version_file_path() -> Path:
    version_files = list(Path(".").rglob("_version.py"))
    if len(version_files) == 1:
        return version_files[0]
    elif len(version_files) > 1:
        raise MultipleFilesFoundError(version_files)

    _help = "Run the following: echo '__version__ = \"1.0.0\"' > _version.py\n"
    raise FileNotFoundError(f"Could not find _version.py. {_help}")


def extract_version_from_file(pyversion_file: Path) -> str:
    version_file_content = pyversion_file.read_text()
    match = re.compile(r'__version__ = "(?P<version>\d+\.\d+\.\d+)"').match(
        version_file_content
    )
    if not match:
        raise ValueError("Could not find version in _version.py")

    version = match.groupdict()["version"]
    return version


def update_version_file(pyversion_file: Path, version: str, new_version: str) -> None:
    current_content = pyversion_file.read_text()
    new_content = current_content.replace(
        f'__version__ = "{version}"', f'__version__ = "{new_version}"'
    )
    pyversion_file.write_text(new_content)
    
def tag(file:Path, version:str, message: str) -> None:
    # Check if version was updated/is waiting to be committed
    proc = subprocess.run(f"git status {file}", shell=True, capture_output=True)
    if "nothing to commit, working tree clean" in proc.stdout.decode():
        raise RuntimeError("Version was not updated for some reason.")
    # Add, commit, tag . Return code 0 means success
    assert 0 == subprocess.call(f"git add {file}", shell=True)
    assert 0 == subprocess.call(f"git commit -m \"chore: version updated\"", shell=True)
    assert 0 == subprocess.call(f"git tag {version}", shell=True)
    



def update_version_info(version_info: list[int], identifier: str) -> list[int]:
    if identifier not in Identifiers.names():
        raise ValueError(
            f"Expected one of {Identifiers.names_repr()}. Got: {identifier.lower()!r}"
        )

    if identifier.lower() in Identifiers.MAJOR:
        version_info[0] += 1
        version_info[1] = 0
        version_info[2] = 0
    elif identifier.lower() in Identifiers.MINOR:
        version_info[1] += 1
        version_info[2] = 0
    elif identifier.lower() in Identifiers.PATCH:
        version_info[2] += 1

    return version_info


def version2info(version: str) -> list[int]:
    return list(map(int, version.split(".")))


def info2version(info: list[int]) -> str:
    return ".".join(list(map(str, info)))



VERSION_FILE = find_version_file_path()

version = extract_version_from_file(VERSION_FILE)
version_info = version2info(version)

new_version_info = update_version_info(version_info, "fix")
new_version = info2version(new_version_info)

update_version_file(VERSION_FILE, version, new_version)


In [105]:
msg="""
fix: something
feat: other
chore: version updated
"""

def prepare_tag_message(msg: str) -> str:
    msg_lines = msg.split("\n")
    msg_lines = [line.strip() for line in msg_lines if line.strip()]
    msg = " ".join(f'-m "{line}"' for line in msg_lines)
    return msg

prepare_tag_message(msg)

'-m "fix: something" -m "feat: other" -m "chore: version updated"'

In [161]:
import argparse

def get_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(
        description="Update version and tag it with a message"
    )
    
    sub_parser = parser.add_subparsers(dest="action", required=True)
    update_parser = sub_parser.add_parser("update")
    update_parser.add_argument(
        "identifier",
        type=str,
        choices=Identifiers.names(),
        help="Which part of the version to update",
    )
    update_parser.add_argument(
        "-t",
        "--tag",
        type=str,
        nargs="?",
        const="",
        default=None,
        dest="tag_message",
        help="Tag the version with a message",
    )
    tag_parser = sub_parser.add_parser("tag")
    tag_parser.add_argument(
        "tag_message",
        type=str,
        nargs="?",
        default="",
        help="Tag the version with a message",
    )
    
    return  parser
    

def get_args() -> argparse.Namespace:
    parser = get_parser()
    # args = parser.parse_args(["update","fix"])
    # args = parser.parse_args(["update","fix", "--tag"])
    # args = parser.parse_args(["update","fix","--tag","feat: something"])
    # args = parser.parse_args(["tag"])
    args = parser.parse_args(["tag","feat: something"])
    return args

get_args()

# parser = get_parser()
# for act in parser._actions:
#     print(act.dest)

Namespace(action='tag', tag_message='feat: something')

True

In [97]:
import subprocess

def tag(file:Path, version:str, message: str) -> None:
    # Check if version was updated/is waiting to be committed
    proc = subprocess.run(f"git status {file}", shell=True, capture_output=True)
    if "nothing to commit, working tree clean" in proc.stdout.decode():
        raise RuntimeError("Version was not updated for some reason.")
    # Add, commit, tag . Return code 0 means success
    assert 0 == subprocess.call(f"git add {file}", shell=True)
    assert 0 == subprocess.call(f"git commit -m \"chore: version updated\"", shell=True)
    assert 0 == subprocess.call(f"git tag {version}", shell=True)
    

tag(VERSION_FILE, new_version, "Version updated.")

[master c240461] chore: version updated
 2 files changed, 44 insertions(+), 18 deletions(-)
